In [80]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [81]:
df1 = pd.read_csv("diabetes.csv")
df2 = pd.read_csv("heart.csv")

In [82]:
print(df1.head())
print(df1.info())
print(df1.describe())

print(df2.head())
print(df2.info())
print(df2.describe())

   Diabetes_012  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0           0.0     1.0       1.0        1.0  40.0     1.0     0.0   
1           0.0     0.0       0.0        0.0  25.0     1.0     0.0   
2           0.0     1.0       1.0        1.0  28.0     0.0     0.0   
3           0.0     1.0       0.0        1.0  27.0     0.0     0.0   
4           0.0     1.0       1.0        1.0  24.0     0.0     0.0   

   HeartDiseaseorAttack  PhysActivity  Fruits  ...  AnyHealthcare  \
0                   0.0           0.0     0.0  ...            1.0   
1                   0.0           1.0     0.0  ...            0.0   
2                   0.0           0.0     1.0  ...            1.0   
3                   0.0           1.0     1.0  ...            1.0   
4                   0.0           1.0     1.0  ...            1.0   

   NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex   Age  Education  \
0          0.0      5.0      18.0      15.0       1.0  0.0   9.0        4.0   
1     

In [83]:
# checking missing values
print(df1.isnull().sum())
print(df2.isnull().sum())

Diabetes_012            0
HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
dtype: int64
HeartDisease        0
BMI                 0
Smoking             0
AlcoholDrinking     0
Stroke              0
PhysicalHealth      0
MentalHealth        0
DiffWalking         0
Sex                 0
AgeCategory         0
Race                0
Diabetic            0
PhysicalActivity    0
GenHealth           0
SleepTime           0
Asthma              0
KidneyDisease       0
SkinCancer          0
dtype: int64


In [84]:
# Handle Missing Values
# Both datasets are reported to have no missing values.

# Schema matching
# Renaming columns 
df1.rename(columns={'HvyAlcoholConsump':'AlcoholDrinking', 'Diabetes_012': 'Diabetic', 'HeartDiseaseorAttack': 'HeartDisease'}, inplace=True)
df2.rename(columns={'AgeCategory': 'Age', 'Smoking': 'Smoker', 'PhysicalHealth': 'PhysHlth', 'MentalHealth': 'MentHlth', 'GenHealth': 'GenHlth', 'DiffWalking': 'DiffWalk', 'PhysicalActivity': 'PhysActivity'}, inplace=True)


In [85]:
# Convert Data Types
# Convert binary variables to int (if not already)
# List of columns to convert to int
columns_to_convert_df1= df1.columns
columns_to_convert_df2=['BMI','PhysHlth',  'MentHlth']
# Convert each column in the list to int
df1[columns_to_convert_df1] = df1[columns_to_convert_df1].astype(int)
df2[columns_to_convert_df2] = df2[columns_to_convert_df2].astype(int)



In [86]:
df2['Diabetic'] = df2['Diabetic'].replace({'No, borderline diabetes': 'No'})
df2['Diabetic'] = df2['Diabetic'].replace({'Yes (during pregnancy)': 'Yes'})
yes_no_columns = ['HeartDisease','Smoker', 'AlcoholDrinking', 'DiffWalk', 'Diabetic', 'PhysActivity']

# Automatically convert all columns with "Yes" and "No" values
for column in df2.columns:
    if set(df2[column].unique()) == {'Yes', 'No'}:
        df2[column] = df2[column].map({'Yes': 1, 'No': 0})

# Convert GenHlth to integers (poor/fair/good/very good/excellent -> 5/4/3/2/1)
gen_health_mapping = {'Poor': 5, 'Fair': 4, 'Good': 3, 'Very good': 2, 'Excellent': 1}
df2['GenHlth'] = df2['GenHlth'].map(gen_health_mapping)

# Convert Sex to integers (0=female, 1=male)
df2['Sex'] = df2['Sex'].map({'Female': 0, 'Male': 1})


In [87]:
# Function to calculate the mean of age ranges
def mean_of_range(age_range):
    if age_range == '80 or older':
        # Assuming 85 as the upper limit for the '80 or older' category
        return 82.5
    else:
        # Split the range into a list of the lower and upper bounds, convert to integers, and calculate the mean
        lower, upper = map(int, age_range.split('-'))
        return (lower + upper) / 2

# Apply the function to the Age column
df2['Age'] = df2['Age'].apply(mean_of_range)
df2['Age'] = df2['Age'].astype(int)

In [88]:

# # List of numerical columns to standardize (adjust as needed)
# numerical_cols_df1 = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth', 'Age']
# numerical_cols_df2 = ['BMI', 'GenHlth', 'PhysHlth', 'MentHlth', 'Age']  # Assuming GenHlth is not available

# # Initialize the StandardScaler
# scaler = StandardScaler()

# # Standardize df1
# df1[numerical_cols_df1] = scaler.fit_transform(df1[numerical_cols_df1])

# # Standardize df2 - Reinitialize the scaler to avoid data leakage between df1 and df2
# scaler = StandardScaler()
# df2[numerical_cols_df2] = scaler.fit_transform(df2[numerical_cols_df2])


In [89]:
# Feature Selection
columns_to_drop_df1 = ['AnyHealthcare', 'NoDocbcCost', 'Education', 'Income','CholCheck', 'Fruits', 'Veggies', 'Stroke','HighBP',  'HighChol']  # Replace 'Column1', 'Column2', etc. with actual column names from df1
columns_to_drop_df2 = ['Race', 'KidneyDisease', 'SkinCancer', 'Stroke','SleepTime',  'Asthma']  # Replace 'Column3', 'Column4', etc. with actual column names from df2

# Drop the specified columns from df1
df1 = df1.drop(columns=columns_to_drop_df1)

# Drop the specified columns from df2
df2 = df2.drop(columns=columns_to_drop_df2)

# Now df1 and df2 have the unwanted columns removed


In [90]:
print(df1.head())
print(df2.head())

   Diabetic  BMI  Smoker  HeartDisease  PhysActivity  AlcoholDrinking  \
0         0   40       1             0             0                0   
1         0   25       1             0             1                0   
2         0   28       0             0             0                0   
3         0   27       0             0             1                0   
4         0   24       0             0             1                0   

   GenHlth  MentHlth  PhysHlth  DiffWalk  Sex  Age  
0        5        18        15         1    0    9  
1        3         0         0         0    0    7  
2        5        30        30         1    0    9  
3        2         0         0         0    0   11  
4        2         3         0         0    0   11  
   HeartDisease  BMI  Smoker  AlcoholDrinking  PhysHlth  MentHlth  DiffWalk  \
0             0   16       1                0         3        30         0   
1             0   20       0                0         0         0         0   
2     

In [91]:
df1.to_csv('df1.csv', index=False)
df2.to_csv('df2.csv', index=False)

In [92]:

# Assuming df1 and df2 are your DataFrames

# Align the columns in df1 and df2 so they are in the same order
common_cols = ['Diabetic', 'HeartDisease', 'BMI', 'Sex', 'Age', 'Smoker', 'AlcoholDrinking', 'PhysHlth', 
               'MentHlth', 'GenHlth', 'DiffWalk',  'PhysActivity']
unique_cols_df1 = [col for col in df1.columns if col not in common_cols]
unique_cols_df2 = [col for col in df2.columns if col not in common_cols]

# Reorder df1 and df2 to have common columns first, then unique columns
df1_reordered = df1[common_cols + unique_cols_df1]
df2_reordered = df2[common_cols + unique_cols_df2]

# Concatenate df1 and df2
merged_df = pd.concat([df1_reordered, df2_reordered], axis=0, ignore_index=True, sort=False)

print(merged_df.head())


   Diabetic  HeartDisease  BMI  Sex  Age  Smoker  AlcoholDrinking  PhysHlth  \
0         0             0   40    0    9       1                0        15   
1         0             0   25    0    7       1                0         0   
2         0             0   28    0    9       0                0        30   
3         0             0   27    0   11       0                0         0   
4         0             0   24    0   11       0                0         0   

   MentHlth  GenHlth  DiffWalk  PhysActivity  
0        18        5         1             0  
1         0        3         0             1  
2        30        5         1             0  
3         0        2         0             1  
4         3        2         0             1  


In [93]:
# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_dataset2.csv', index=False)

In [16]:
print(merged_df.columns)


Index(['Diabetic', 'HeartDisease', 'BMI', 'Sex', 'Age', 'Smoker',
       'AlcoholDrinking', 'Stroke', 'PhysHlth', 'MentHlth', 'GenHlth',
       'DiffWalk', 'PhysActivity', 'HighBP', 'HighChol', 'CholCheck', 'Fruits',
       'Veggies', 'SleepTime', 'Asthma'],
      dtype='object')


In [94]:
df = pd.read_csv("merged.csv")
# Define the features you want to normalize
features_to_normalize = ['BMI', 'Age', 'PhysHlth', 'MentHlth', 'GenHlth']

# Instantiate the scaler
scaler = StandardScaler()

# Fit and transform the features
df[features_to_normalize] = scaler.fit_transform(df[features_to_normalize])

# Print the normalized DataFrame
print(df.head())

   Diabetic  HeartDisease       BMI  Sex       Age  Smoker  AlcoholDrinking  \
0         0             0  1.840433    0 -0.932038       1                0   
1         0             0 -0.475032    0 -1.006755       1                0   
2         0             0 -0.011939    0 -0.932038       0                0   
3         0             0 -0.166303    0 -0.857321       0                0   
4         0             0 -0.629396    0 -0.857321       0                0   

   PhysHlth  MentHlth   GenHlth  DiffWalk  PhysActivity  
0  1.352954  1.865562  2.413695         1             0  
1 -0.452063 -0.463593  0.519080         0             1  
2  3.157972  3.418332  2.413695         1             0  
3 -0.452063 -0.463593 -0.428228         0             1  
4 -0.452063 -0.075400 -0.428228         0             1  


In [95]:
df.to_csv('merged1.csv', index=False)

In [96]:
print(df.columns)

Index(['Diabetic', 'HeartDisease', 'BMI', 'Sex', 'Age', 'Smoker',
       'AlcoholDrinking', 'PhysHlth', 'MentHlth', 'GenHlth', 'DiffWalk',
       'PhysActivity'],
      dtype='object')
